# Artificial Intelligence — Lab — Exercise 05

#### 6 October 2020

## Davis–Putnam–Logemann–Loveland (DPLL) algorithm

Implement David-Putnam algorithm for checking the satisfiability of a sentence represented in propositional logic.
<hr>

The Davis–Putnam–Logemann–Loveland (DPLL) algorithm is a complete, 
backtracking-based search algorithm for deciding the satisfiability of propositional logic formulae 
in conjunctive normal form, i.e. for solving the CNF-SAT problem.
<hr>

Theory is adapted from Artificial Intelligence: A Modern Approach by Peter Norvig & Stuart J. Russell

<hr>
This algorithm is very similar to Backtracking-Search. It recursively enumerates possible models in a depth-first fashion with the following improvements over algorithms like Truth Table Entailment.

1. Early termination:
In certain cases, the algorithm can detect the truth value of a statement using just a partially completed model. For example, $(P\lor Q)\land(P\lor R)$ is true if P is true, regardless of other variables. This reduces the search space significantly.
<br><br>
2. Pure symbol heuristic:
A symbol that has the same sign (positive or negative) in all clauses is called a pure symbol. It isn't difficult to see that any satisfiable model will have the pure symbols assigned such that its parent clause becomes true. For example, $(P\lor\neg Q)\land(\neg Q\lor\neg R)\land(R\lor P)$ has P and Q as pure symbols and for the sentence to be true, P has to be true and Q has to be false. The pure symbol heuristic thus simplifies the problem a bit.
<br><br>
3. Unit clause heuristic:
In the context of DPLL, clauses with just one literal and clauses with all but one false literals are called unit clauses. If a clause is a unit clause, it can only be satisfied by assigning the necessary value to make the last literal true. We have no other choice.
<br><br>
Assigning one unit clause can create another unit clause. For example, when P is false, $(P\lor Q)$ becomes a unit clause, causing true to be assigned to Q. A series of forced assignments derived from previous unit clauses is called unit propagation. In this way, this heuristic simplifies the problem further.
<hr>


In [1]:
class DPLL:
    true_literals = set()
    false_literals = set()
    props, branches = 0, 0
    cnf = []
    literals = []
    
    def __init__(self, formula):
        """To initialize the DPLL Solver with the required CNF Form & Literals. """
        
        self.literals = [literal for literal in list(set(formula)) if literal.isalpha()]
        self.cnf = formula.split(", ")
        print("Initial Formula:\t", self)
        
        
    def __str__(self):
        """To convert the Pythonic-list used for CNF notation into a readable formula in CNF. """
        
        cnf_str = ""
        
        for clause in self.cnf:  #represent each claue in CNF
            if len(clause) > 0:
                cnf_str += '(' + clause.replace(' ', ' ∨ ') + ') ∧ '
    
        if cnf_str == "":       #formula is empty
            cnf_str = "()"
            
        if cnf_str[-2] == "∧":  #removing the final '∧' added which is un-wanted.
            cnf_str = cnf_str[:-2:] 
    
        return cnf_str
    
    
    def DPLL_Algorithm(self):
        """Perform the David-Putnam-Logemann-Loveland algorithm on the given formula. """
        
        new_true_literals  =  []
        new_false_literals =  []
        
        print("\nCurrent Formula:\t", self)
        
        self.true_literals = set(self.true_literals)
        self.false_literals = set(self.false_literals)
        
        self.branches += 1  #Since this is a new branch
        
        cnf = list(set(self.cnf))
        unit_clauses = [clause for clause in cnf if len(clause) < 3]   #clause of len < 3 is a unit clause
        unit_clauses = list(set(unit_clauses))
        
        if unit_clauses:  #Assign truth values to unit clauses for satisfying the given formula
            for unit in unit_clauses:
                self.props += 1
                
                if '¬' in unit:  #False unit clause
                    self.false_literals.add(unit[-1])
                    new_false_literals.append(unit[-1])
                    i = 0
                    
                    while True:
                        if unit in cnf[i]:  #Remove the unit clause from the formula
                            cnf.remove(cnf[i])
                            i -= 1
                        
                        elif unit[-1] in cnf[i]:  #If a ¬(unit clause) exists in the formula
                            cnf[i] = cnf[i].replace(unit[-1], '').strip()
                    
                        i += 1
                    
                        if i >= len(cnf):
                            break
        
                else:  #True unit clause
                    self.true_literals.add(unit)
                    new_true_literals.append(unit)
                    i = 0
                
                    while True:
                        if '¬' + unit in cnf[i]:  #If a ¬(unit clause) exists in the formula
                            cnf[i] = cnf[i].replace('¬' + unit, '').strip()
                        
                            if '  ' in cnf[i]:  #Empty clause
                                cnf[i] = cnf[i].replace('  ', ' ')
                        
                        elif unit in cnf[i]:  #Remove the unit clause from the formula
                            cnf.remove(cnf[i])
                            i -= 1
                            
                        i += 1
                        
                        if i >= len(cnf):
                            break
        
        self.cnf = cnf
        
        print("Current Unit Clauses:\t", unit_clauses)
        print("CNF after Propagation:\t", self)
        
        if len(self.cnf) == 0:  #Formula becomes an empty clause
            return True
        
        if sum(len(clause) == 0 for clause in cnf):  #If there are empty parantheses, indicating contradiction
            
            for literal in new_true_literals:   #clear every newly set true literal
                self.true_literals.remove(literal)
            
            for literal in new_false_literals:  #clear every newly set false literal
                self.false_literals.remove(literal)
                
            print("\nNull clause found. Backtracking...")
            
            return False  #backtrack, since we cannot simplify anymore, and it is unsatisfiable in the current branch
        
        self.literals = [literal for literal in list(set(''.join(self.cnf))) if literal.isalpha()]
        #form the literals again from the updated formula
        
        first_literal = self.literals[0]  #choose the first literal to set as True/False to find satisfying assignments
        
        self.cnf = cnf + [first_literal]  #Try with first literal set to True
        print("\nTrying with {0} as True...".format(first_literal))
        
        if self.DPLL_Algorithm():  #recursively work on the new formula
            return True
        
        self.cnf = cnf + ['¬' + first_literal]  #Try with first literal set to False (i.e ¬(first literal) is True)
        print("\nTrying with ¬{0} as True...".format(first_literal))
        
        if self.DPLL_Algorithm():  #recursively work on the new formula
            return True
        
        
        else:  #no satisfying assignments were found to the literals of the formula
            self.cnf = cnf
            
            for literal in new_true_literals:   #remove all the assignments
                self.true_literals.remove(literal)
            
            for literal in new_false_literals:  #remove all the assignments
                self.false_literals.remove(literal)
            
            return False  #the formula is unsatisfiable.
        
    
    def print_result(self, satisfiability):
        """Print the result of the DPLL Algorithm on the formula along with its literals' assignments and statistics. """
        
        if satisfiability == True:
            print("\nThe given CNF statement is satisfiable.")
            print("\nSolution: ")
            
            for literal in self.true_literals:
                print("\t"+literal, "= True")
                
            for literal in self.false_literals:
                print("\t"+literal, "= False")
            
        else:
            print("\nThe given CNF statement was unsatisfiable.")
            
        print("\nNo. of branches:\t\t", self.branches)
        print("\nNo. of unit propagations:\t", self.props)


<hr>
Running DPLL Algorithm for $(X \lor Y \lor Z)\land(X \lor\neg Y \lor Z)\land(\neg X \lor Y \lor\neg Z)\land(\neg Z)$
<hr>

In [2]:
dpll_solver = DPLL("X Y Z, X ¬Y Z, ¬X Y ¬Z, ¬Z")
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Initial Formula:	 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y ∨ Z) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 

Current Formula:	 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y ∨ Z) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z) 
Current Unit Clauses:	 ['¬Z']
CNF after Propagation:	 (X ∨ Y) ∧ (X ∨ ¬Y) 

Trying with Y as True...

Current Formula:	 (X ∨ Y) ∧ (X ∨ ¬Y) ∧ (Y) 
Current Unit Clauses:	 ['Y']
CNF after Propagation:	 (X) 

Trying with X as True...

Current Formula:	 (X) ∧ (X) 
Current Unit Clauses:	 ['X']
CNF after Propagation:	 ()

The given CNF statement is satisfiable.

Solution: 
	Y = True
	X = True
	Z = False

No. of branches:		 3

No. of unit propagations:	 3


<hr>
Running DPLL Algorithm for $(\neg P \lor Q)\land(\neg Q \lor\neg R)\land(\neg S \lor P)\land(P \lor Q \lor\neg R)$
<hr>

In [3]:
dpll_solver = DPLL("¬P Q, ¬Q ¬R, R ¬S, ¬S P, P Q ¬R")
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Initial Formula:	 (¬P ∨ Q) ∧ (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) 

Current Formula:	 (¬P ∨ Q) ∧ (¬Q ∨ ¬R) ∧ (R ∨ ¬S) ∧ (¬S ∨ P) ∧ (P ∨ Q ∨ ¬R) 
Current Unit Clauses:	 []
CNF after Propagation:	 (¬S ∨ P) ∧ (¬Q ∨ ¬R) ∧ (P ∨ Q ∨ ¬R) ∧ (¬P ∨ Q) ∧ (R ∨ ¬S) 

Trying with R as True...

Current Formula:	 (¬S ∨ P) ∧ (¬Q ∨ ¬R) ∧ (P ∨ Q ∨ ¬R) ∧ (¬P ∨ Q) ∧ (R ∨ ¬S) ∧ (R) 
Current Unit Clauses:	 ['R']
CNF after Propagation:	 (¬S ∨ P) ∧ (¬Q) ∧ (P ∨ Q) ∧ (¬P ∨ Q) 

Trying with Q as True...

Current Formula:	 (¬S ∨ P) ∧ (¬Q) ∧ (P ∨ Q) ∧ (¬P ∨ Q) ∧ (Q) 
Current Unit Clauses:	 ['¬Q', 'Q']
CNF after Propagation:	 (¬S ∨ P) ∧ (¬P) ∧ (P) 

Null clause found. Backtracking...

Trying with ¬Q as True...

Current Formula:	 (¬S ∨ P) ∧ (¬Q) ∧ (P ∨ Q) ∧ (¬P ∨ Q) ∧ (¬Q) 
Current Unit Clauses:	 ['¬Q']
CNF after Propagation:	 (P) ∧ (¬S ∨ P) ∧ (¬P) 

Trying with S as True...

Current Formula:	 (P) ∧ (¬S ∨ P) ∧ (¬P) ∧ (S) 
Current Unit Clauses:	 ['¬P', 'S', 'P']
CNF after Propagation:	 ()

Null clause found.

<hr>
Custom formula : Separate clauses in formula by commas ", " and literals inside a clause with spaces. Use "¬" for negation of a literal.
<hr>

In [4]:
dpll_solver = DPLL(input("Enter a formula in CNF: "))
satisfiability = dpll_solver.DPLL_Algorithm()
dpll_solver.print_result(satisfiability)

Enter a formula in CNF: P, ¬P
Initial Formula:	 (P) ∧ (¬P) 

Current Formula:	 (P) ∧ (¬P) 
Current Unit Clauses:	 ['¬P', 'P']
CNF after Propagation:	 ()

Null clause found. Backtracking...

The given CNF statement was unsatisfiable.

No. of branches:		 1

No. of unit propagations:	 2
